# Big G Express: Predicting Derates
In this project, you will be working with fault code data and vehicle onboard diagnostic data to try and predict an upcoming full derate. These are indicated by an SPN 5246. 

You have been provided with a two files containing the data you will use to make these predictions (J1939Faults.csv and VehicleDiagnosticOnboardData.csv) as well as two files describing some of the contents (DataInfo.docx and Service Fault Codes_1_0_0_167.xlsx) 

Note that in its raw form the data does not have "labels", so you must define what labels you are going to use and create those labels in your dataset. Also, you will likely need to perform some significant feature engineering in order to build an accurate predictor.

There are service locations at (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722), so you should remove any records in the vicinity of these locations, as fault codes may be tripped when working on the vehicles.

When evaluating the performance of your model, assume that the cost associated with a missed full derate is approximately $4000.00$ in towing and repairs, and the cost of a false positive prediction is about $500 due to having the truck off the road and serviced unnecessarily.

##### Why Derate?
A derate is designed to protect you from causing engine or DPF damage. A derate can be caused by various sensors or systems. However, when it's a specific 5 MPH derate then you can be certain that the problem is related to the emissions, aka the aftertreatment system.

ref:
https://otrperformance.com/blogs/quick-tips/your-cummins-has-a-5-mph-def-derate-spn-5246-what-now

A failed component is usually what triggers this code.

Common Failures

 * Failed DEF doser valve
 * Associated fault code: SPN 5394
 * You ran out of DEF fluid
 * Associated fault code: SPN 5392, SPN 1761
 * Inlet and Outlet NOx sensors failed, not making pressure
 * Associated fault code: SPN 4094
 * EGR system malfunction causing NOX efficiency problems
 * DEF pump failed, not making pressure
 * Associated fault code: SPN 4334, SPN 4339
 * DEF module has failed or DEF harness failure or no power to DEF module causes DEF gauge to be empty and showing datalink error and SCR malfunction.
 * The DEF / ECM could also need updating to eliminate ghost codes.

#### Data exploration

Looking at the first record, here is a breakdown of the important values.

* ESS_Id, actionDescription, ecuSoftwareVersion, ecuSerialNumber, ecuModel, ecuMake, ecuSource, faultValue, and MCTNumber are unlikely to provide any predictive value.
* We can see the time of the event in the **EventTimeStamp** column. Note that this may be different from the **LocationTimeStamp** value, which indicates when the Latitude/Longitude values were recorded.
* The **spn** and **fmi** columns together indicate the type of fault, and there may be a description of that fault in the **eventDescription** column, although this column is sometimes missing.
* Faults are recorded when the light goes on and when it goes off, which is indicated by the **active** column, with True indicating the light turning on and False indicating turning off. The number of times the code has been set or unset is in the **faultValue** column, although this value can be unreliable. 
* Each truck has an identifier, the **EquipmentID** value.
* Each record can be linked to the on-board diagnostics data through the **RecordID** column.

To get the on-board diagnostics at the time of the fault code, we can match the RecordID to the FaultId.

#### Basic things/Next Steps:
    1. MT and PS push to github
    2. Any Q's for Michael
        2a. How much to cut out around service center? (less than 10, more like 1-2 miles seems to be consensus)
    3. Convert from lat/lon to geopandas and make a circular fence around the service centers
    4. Start by looking at one truck with appropriate derate codes (faults[((faults.spn==5246)) | ((faults.spn==1569)&(faults.fmi==31))].count())


#### Strategic/Tactical considerations:
    1. Need to look at both the cost of derates and the cost of a false positive
    2. Perhaps focusing on features in the emissions, aka the aftertreatment system.

##### Ideas:
  *  Start with just one truck...
  *  Pull out EquipmentID, RecordID, timestamp on every fault code that has a derate.
  *  Then look at all fault codes that are timestamped between 2 and 24 hours before, pulling ID data from there.
  *      (You might have to create your own unique index?)
  *  Then, run through diagnostics, pulling all data with correct RecordID
  *  Possible interesting columns:
  *      Number of records
  *      Number of faults
  *      Highly recurring faults(like top 10 or 20)
  
  * QUESTION: How to incorporate false positives? via confusion_matrix? Note the weighting that can be used in the datacamp intermediate pipeline course...